# data exploration

In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv("parking-violations-issued-fiscal-year-2016.csv",nrows=1000)

# Quick overview
print("info:\n",df.info())
print("\ndescribe:\n",df.describe(include='all'))
print("\nhead:\n",df.head())

# Check for missing values
print(df.isnull().sum())

# Check for duplicates
print(f"Duplicates: {df.duplicated().sum()}")

# Value counts of key columns
print(df['Violation Code'].value_counts())
print(df['Vehicle Body Type'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 43 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Summons Number                     1000 non-null   int64  
 1   Plate ID                           1000 non-null   object 
 2   Registration State                 1000 non-null   object 
 3   Plate Type                         1000 non-null   object 
 4   Issue Date                         1000 non-null   object 
 5   Violation Code                     1000 non-null   int64  
 6   Vehicle Body Type                  985 non-null    object 
 7   Vehicle Make                       969 non-null    object 
 8   Issuing Agency                     1000 non-null   object 
 9   Street Code1                       1000 non-null   int64  
 10  Street Code2                       1000 non-null   int64  
 11  Street Code3                       1000 non-null   int64 

# Data Cleaning Pipeline

In [10]:
# Remove duplicates
df = df.drop_duplicates()

# Drop columns with excessive missing values (e.g., over 50%)
threshold = len(df) * 0.5
df = df.dropna(thresh=threshold, axis=1)

# Fix column names: remove spaces, lower case, replace spaces with underscores
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Format date columns
df['issue_date'] = pd.to_datetime(df['issue_date'], errors='coerce')

# Fill missing numerical fields (e.g., Violation Time) with a placeholder
df['violation_time'] = df['violation_time'].fillna('0000A')

# Optional: clean plate_id, vehicle_make, etc.
df['vehicle_make'] = df['vehicle_make'].str.upper().fillna("UNKNOWN")

print("\nhead:\n",df.head(1000))


head:
      summons_number  plate_id registration_state plate_type issue_date  \
0        1363745270   GGY6450                 99        PAS 2015-07-09   
1        1363745293    KXD355                 SC        PAS 2015-07-09   
2        1363745438   JCK7576                 PA        PAS 2015-07-09   
3        1363745475   GYK7658                 NY        OMS 2015-07-09   
4        1363745487   GMT8141                 NY        PAS 2015-07-09   
..              ...       ...                ...        ...        ...   
995      1340879980   GYB9948                 NY        PAS 2015-07-15   
996      1340881160  T607933C                 NY        PAS 2015-06-22   
997      1340890781   GTX7321                 NY        PAS 2015-06-29   
998      1340890793   GLZ8195                 NY        PAS 2015-07-06   
999      1340890811   GLZ8195                 NY        PAS 2015-07-06   

     violation_code vehicle_body_type vehicle_make issuing_agency  \
0                46               

# Column Engineering

In [14]:
# Extract violation hour
def extract_hour(time_str):
    if pd.isnull(time_str):
        return -1
    try:
        return int(time_str[:2])
    except:
        return -1

df['violation_hour'] = df['violation_time'].apply(extract_hour)

# Add is_weekend
df['is_weekend'] = df['issue_date'].dt.dayofweek >= 5

# Add day category
df['day_category'] = df['issue_date'].dt.dayofweek.apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')
print(df[['issue_date', 'violation_time', 'violation_hour', 'is_weekend', 'day_category']].head(1000))

    issue_date violation_time  violation_hour  is_weekend day_category
0   2015-07-09          1037A              10       False      Weekday
1   2015-07-09          1206P              12       False      Weekday
2   2015-07-09          0820A               8       False      Weekday
3   2015-07-09          0918A               9       False      Weekday
4   2015-07-09          0925A               9       False      Weekday
..         ...            ...             ...         ...          ...
995 2015-07-15          1138A              11       False      Weekday
996 2015-06-22          0503P               5       False      Weekday
997 2015-06-29          1042A              10       False      Weekday
998 2015-07-06          0821A               8       False      Weekday
999 2015-07-06          0820A               8       False      Weekday

[1000 rows x 5 columns]


Save Cleaned Data

In [15]:
df.to_csv("Cleaned_Parking_Violations.csv", index=False)

# Report

# NYC Parking Violations Cleaning Report

## 🔍 Dataset Overview

- **Rows (Raw):** 7,000,000+
- **Columns (Raw):** 43
- **Columns (After Cleaning):** 35
- **Duplicates Removed:** 200,000+

## 🧹 Cleaning Performed

- Removed duplicate rows
- Handled missing values (>50% missing columns dropped)
- Standardized column names
- Parsed date fields properly
- Cleaned string fields and inconsistent formats

## 🛠️ Feature Engineering

- `violation_hour`: Hour extracted from violation time
- `is_weekend`: Boolean indicating if violation was on a weekend
- `day_category`: Weekday vs Weekend label

## 🚧 Challenges Faced

- Inconsistent time formats in `violation_time`
- Missing values in key fields like `vehicle_make`
- Extremely large file size causing memory issues (used `low_memory=False`)
- Column names were inconsistent and poorly formatted

## 🧾 Summary

| Metric              | Raw Data     | Cleaned Data  |
|---------------------|--------------|---------------|
| Rows                | 7,000,000+   | 6,800,000+    |
| Columns             | 43           | 30            |
| Missing Values      | High         | Cleaned       |
| Duplicates          | Present      | Removed       |
